## Az eredményeket mentsd a src/weekly modul-ba weekly_test_6.py néven

#### Használható modulok: pathlib, pandas, typing, str, statsmodels

In [26]:
from pathlib import Path
!pip3 install fastparquet
import fastparquet

  Using cached fastparquet-2023.8.0-cp310-cp310-macosx_11_0_arm64.whl (681 kB)
  Using cached cramjam-2.7.0-cp310-cp310-macosx_10_9_x86_64.macosx_11_0_arm64.macosx_10_9_universal2.whl (3.2 MB)


In [16]:
datalib = Path.cwd().parent.joinpath('data')

In [17]:
import pandas as pd
import statsmodels

In [18]:
"""
1., Olvasd be a data mappa sp500.parquet nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

'\n1., Olvasd be a data mappa sp500.parquet nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet\n'

In [30]:
sp500=pd.read_parquet('../data/sp500.parquet')
sp500.head()

,Symbol,Monthly Returns
Date,,
2000-02-29,A,0.569405
2000-03-31,A,0.001203
2000-05-31,A,-0.169252
2000-06-30,A,0.001698
2000-07-31,A,-0.447458


In [ ]:
"""
2., Olvasd be az ff_factors.parquet fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen fastparquet
"""

In [31]:
ff_factors=pd.read_parquet('../data/ff_factors.parquet')
ff_factors.head()

,Mkt-RF,SMB,HML,RF
Date,,,,
1926-07-31,0.0296,-0.0256,-0.0243,0.022
1926-08-31,0.0264,-0.0117,0.0382,0.025
1926-09-30,0.0036,-0.0140,0.0013,0.023
1926-10-31,-0.0324,-0.0009,0.0070,0.032
1926-11-30,0.0253,-0.0010,-0.0051,0.031


In [ ]:
"""
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a 'Date' elsődleges kulcs alapján.
"""

In [35]:
data=pd.merge(sp500, ff_factors, on='Date', how='left')
data.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF
Date,,,,,,
2000-02-29,A,0.569405,0.0245,0.2136,-0.0959,0.043
2000-03-31,A,0.001203,0.0520,-0.1720,0.0813,0.047
2000-05-31,A,-0.169252,-0.0442,-0.0605,0.0475,0.050
2000-06-30,A,0.001698,0.0464,0.1284,-0.0842,0.040
2000-07-31,A,-0.447458,-0.0251,-0.0307,0.0831,0.048


![Joined data](../resources/weekly6/joined_data.jpg)

In [ ]:
"""
4., Készíts egy új 'Excess Return' nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
"""

In [36]:
data['Excess Return']=data['Monthly Returns'] - data['RF']
data.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return
Date,,,,,,,
2000-02-29,A,0.569405,0.0245,0.2136,-0.0959,0.043,0.526405
2000-03-31,A,0.001203,0.0520,-0.1720,0.0813,0.047,-0.045797
2000-05-31,A,-0.169252,-0.0442,-0.0605,0.0475,0.050,-0.219252
2000-06-30,A,0.001698,0.0464,0.1284,-0.0842,0.040,-0.038302
2000-07-31,A,-0.447458,-0.0251,-0.0307,0.0831,0.048,-0.495458


In [ ]:
"""
5., Rendezd sorba dátum szerint az adatokat, majd generálj egy új oszlopot ('ex_ret_1'), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a következő időszaki Excess Return érték. 
"""

![new column](../resources/weekly6/ex_ret_1.jpg)

In [58]:
data.sort_index(inplace=True)
data.head()
data['ex_ret_1']=data.groupby('Symbol')['Excess Return'].shift(-1)
data.sort_values(['Symbol', 'Date'], ascending=[True, True], inplace=True)
data.head()

,Symbol,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return,ex_ret_1
Date,,,,,,,,
2000-02-29,A,0.569405,0.0245,0.2136,-0.0959,0.043,0.526405,-0.045797
2000-03-31,A,0.001203,0.0520,-0.1720,0.0813,0.047,-0.045797,-0.219252
2000-05-31,A,-0.169252,-0.0442,-0.0605,0.0475,0.050,-0.219252,-0.038302
2000-06-30,A,0.001698,0.0464,0.1284,-0.0842,0.040,-0.038302,-0.495458
2000-07-31,A,-0.447458,-0.0251,-0.0307,0.0831,0.048,-0.495458,0.446933


In [59]:
"""
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer törlöd az össze olyan sort, amely az 'ex_ret_1' oszlopban hiányos, majd ezt követően, törlöd az összes olyan sort, ami a 'HML' oszlopban hiányos
"""

"\n6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer törlöd az össze olyan sort, amely az 'ex_ret_1' oszlopban hiányos, majd ezt követően, törlöd az összes olyan sort, ami a 'HML' oszlopban hiányos\n"

In [62]:
data.dropna(subset=['ex_ret_1'], inplace=True)
data.dropna(subset=['HML'], inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 86113 entries, 2000-02-29 to 2022-10-31
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Symbol           86113 non-null  object 
 1   Monthly Returns  86113 non-null  float64
 2   Mkt-RF           86113 non-null  float64
 3   SMB              86113 non-null  float64
 4   HML              86113 non-null  float64
 5   RF               86113 non-null  float64
 6   Excess Return    86113 non-null  float64
 7   ex_ret_1         86113 non-null  float64
dtypes: float64(7), object(1)
memory usage: 5.9+ MB


In [ ]:
"""
A meglévő adatokból válaszd ki a Amazon részvényhez tartozó sorokat (AMZN) és töröld a tickereket tartalmazó oszlopot

7., Készíts egy új LinearRegressionSM elnevezésű osztályt. Definiáld benne a __init__ nevű függvényt, amely bemenetként 2 DataDrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (left_hand_side), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (right_hand_side).

8., Egésztsd ki az osztályt egy fit metódussal, ami meghívja a statsmodels segítségével OLS alapon becsült modellt illeszt az osztály adattagjaira. A becslésből visszakapott modellt mentse le az osztály _model adattagjába. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között.

9., Egészítsd ki az osztályt egy get_params metódussal, ami visszaadja a becsült modellt béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen 'Beta coefficients'

10., Egészítsd ki az osztályt egy get_pvalues metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: P-values for the corresponding coefficients

11., Egészítsd ki az osztályt egy get_wald_test_result metódussal, ami visszaadja a bemeneti restrikciós mátrix alapján számolt F és p értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: F-value: fvalue, p-value: pvalue, ahol az fvalue és pvalue helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

12., Egészítse ki az osztályt egy get_model_goodness_values metódussal, ami visszadja a módosított R-négyzet, az Akaike és a Bayes információs kritériumok értékét. A visszatérési típus string legyen, a visszaadandó szöveg: Adjusted R-squared: ars, Akaike IC: ak, Bayes IC: by, ahol ars, ak és by helyére 3 tizedesjegyre kerekítve adja meg a hozzájuk tartozó értékeket.

A létrehozott osztályt és a hozzátartozó metódusokat mentsd le a src.linear_regression mappába, a LinearRegressions.py fájlba.
"""

In [67]:
data=data[data['Symbol']=='AMZN']
data.drop(columns=['Symbol'], inplace=True)
data.head()

,Monthly Returns,Mkt-RF,SMB,HML,RF,Excess Return,ex_ret_1
Date,,,,,,,
2000-01-31,-0.984367,-0.0474,0.0577,-0.0188,0.041,-1.025367,0.023796
2000-02-29,0.066796,0.0245,0.2136,-0.0959,0.043,0.023796,-0.074223
2000-03-31,-0.027223,0.0520,-0.1720,0.0813,0.047,-0.074223,-0.174575
2000-05-31,-0.124575,-0.0442,-0.0605,0.0475,0.050,-0.174575,-0.288383
2000-06-30,-0.248383,0.0464,0.1284,-0.0842,0.040,-0.288383,-0.218396


In [65]:
import statsmodels.api as sm

class LinearRegressionSM:
    def __init__(self, left_hand_side, right_hand_side):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self._model = None

    def fit(self):
        X = sm.add_constant(self.right_hand_side)
        model = sm.OLS(self.left_hand_side, X).fit()
        self._model = model

    def get_params(self):
        beta_coefficients = self._model.params
        return pd.Series(beta_coefficients, name='Beta coefficients')

    def get_pvalues(self):
        p_values = self._model.pvalues
        return pd.Series(p_values, name='P-values for the corresponding coefficients')

        def get_wald_test_result(self, constraints):
        wald_test = self._model.wald_test(constraints)
        f_value = float(wald_test.statistic)
        p_value = float(wald_test.pvalue)
        return f'F-value: {f_value:.3f}, p-value: {p_value:.3f}'

    def get_model_goodness_values(self):
        adjusted_r_squared = self._model.rsquared_adj
        aic = self._model.aic
        bic = self._model.bic
        return f'Adjusted R-squared: {adjusted_r_squared:.3f}, Akaike IC: {aic:.3f}, Bayes IC: {bic:.3f}'

    

In [68]:
round(1.333, 2)

1.33